In [110]:
import geopandas as gpd 
import os
import parcelfunks
import importlib
import pandas as pd

In [12]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [7]:
outDir = r'E:/'
dataDir = r'C:\Users\phwh9568\Data\ParcelAtlas'

In [143]:
countiesPath = r'E:/tl_2022_us_county.gpkg'
mhpPath = r'C:\Users\phwh9568\Data\ParcelAtlas\Mobile_Home_Parks\MobileHomeParks.shp' 
blocksPath = r'E:\mobile_homes\blocks.gdb'

In [13]:
mhps = parcelfunks.mhomes_prepper(mhpPath)

In [111]:
counties = gpd.read_file(countiesPath, dtype={'STATEFP':str, 'COUNTYFP':str, 'GEOID':str})
stateFips = pd.read_csv(os.path.join(dataDir,'stateFips.csv'), dtype={'STATEFP':str, 'STATENS':str})
stateFipsDict=dict(zip(stateFips['STATEFP'].to_list(), stateFips['STATE_NAME'].to_list()))


c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: driver GPKG does not support open option DTYPE
  result = ogr_read(


In [14]:
mhps.columns

Index(['MHPID', 'MH_NAME', 'MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP',
       'MH_STATUS', 'MH_COUNTY_FIPS', 'MH_LATITUDE', 'MH_LONGITUDE',
       'MH_UNITS', 'MH_SIZE', 'geometry'],
      dtype='object')

In [15]:
mhpFips = mhps['MH_COUNTY_FIPS'].unique()

In [113]:
countyFips = counties[['STATEFP','GEOID']].copy()

In [114]:
countyFips.sort_values(['GEOID'], inplace=True)

In [120]:
stateFP = countyFips['STATEFP'].to_list()
countyFP = countyFips['GEOID'].to_list()

In [124]:
for s, g in zip(stateFP, countyFP):
    print(s,g)

01 01001
01 01003
01 01005
01 01007
01 01009
01 01011
01 01013
01 01015
01 01017
01 01019
01 01021
01 01023
01 01025
01 01027
01 01029
01 01031
01 01033
01 01035
01 01037
01 01039
01 01041
01 01043
01 01045
01 01047
01 01049
01 01051
01 01053
01 01055
01 01057
01 01059
01 01061
01 01063
01 01065
01 01067
01 01069
01 01071
01 01073
01 01075
01 01077
01 01079
01 01081
01 01083
01 01085
01 01087
01 01089
01 01091
01 01093
01 01095
01 01097
01 01099
01 01101
01 01103
01 01105
01 01107
01 01109
01 01111
01 01113
01 01115
01 01117
01 01119
01 01121
01 01123
01 01125
01 01127
01 01129
01 01131
01 01133
02 02013
02 02016
02 02020
02 02050
02 02060
02 02063
02 02066
02 02068
02 02070
02 02090
02 02100
02 02105
02 02110
02 02122
02 02130
02 02150
02 02158
02 02164
02 02170
02 02180
02 02185
02 02188
02 02195
02 02198
02 02220
02 02230
02 02240
02 02275
02 02282
02 02290
04 04001
04 04003
04 04005
04 04007
04 04009
04 04011
04 04012
04 04013
04 04015
04 04017
04 04019
04 04021
04 04023
04 04025
0

In [93]:
fipsDict = dict(zip(countyFips['STATEFP'].to_list(), countyFips['GEOID'].to_list()))

In [95]:
fipsZip = zip(countyFips['STATEFP'].to_list(), countyFips['GEOID'].to_list())

In [109]:
countyFips

,STATEFP,GEOID
1323,01,01001
1242,01,01003
1829,01,01005
3016,01,01007
853,01,01009
...,...,...
259,72,72151
966,72,72153
1244,78,78010
1462,78,78020


In [ ]:
for sfips, cfips in zip(countyFips['STATEFP'].to_list(), countyFips['GEOID'].to_list()):
    if os.path.exists(os.path.join(outDir,f'State_{sfips}',cfips)) == True:
        countyMHPs = mhps.loc[mhps['MH_COUNTY_FIPS']==cfips].copy()
        if len(countyMHPs) > 0:
            countyMHPs.to_file(os.path.join(outDir,f'State_{sfips}',cfips,f'{cfips}_mhps.gpkg'), driver='GPKG', layer=f'{cfips}_mhps')
            print(cfips)

In [144]:
usBlocks = gpd.read_file(blocksPath, layer='Blocks_WGS84', dtype={'GEOID':str})

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: driver OpenFileGDB does not support open option DTYPE
  result = ogr_read(
c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  result = ogr_read(


In [145]:
usBlocks['COUNTY_FIPS'] = usBlocks['GEOID'].str[0:5]

In [146]:
for sfips, cfips in zip(countyFips['STATEFP'].to_list(), countyFips['GEOID'].to_list()):
    if os.path.exists(os.path.join(outDir,f'State_{sfips}',cfips)) == True:
        countyBlocks = usBlocks.loc[usBlocks['COUNTY_FIPS']==cfips].copy()
        if len(countyBlocks) > 0:
            countyBlocks.to_file(os.path.join(outDir,f'State_{sfips}',cfips,f'{cfips}_blocks.gpkg'), driver='GPKG', layer=f'{cfips}_blocks')

In [141]:
usBlocks.loc[usBlocks['COUNTY_FIPS']=='05121']

,GEOID,SUFFIX,NAME,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,COUNTY_FIPS


In [142]:
len(usBlocks)

682496

In [128]:
for sfips, cfips in zip(countyFips['STATEFP'].to_list(), countyFips['GEOID'].to_list()):
    print(sfips,cfips)

01 01001
01 01003
01 01005
01 01007
01 01009
01 01011
01 01013
01 01015
01 01017
01 01019
01 01021
01 01023
01 01025
01 01027
01 01029
01 01031
01 01033
01 01035
01 01037
01 01039
01 01041
01 01043
01 01045
01 01047
01 01049
01 01051
01 01053
01 01055
01 01057
01 01059
01 01061
01 01063
01 01065
01 01067
01 01069
01 01071
01 01073
01 01075
01 01077
01 01079
01 01081
01 01083
01 01085
01 01087
01 01089
01 01091
01 01093
01 01095
01 01097
01 01099
01 01101
01 01103
01 01105
01 01107
01 01109
01 01111
01 01113
01 01115
01 01117
01 01119
01 01121
01 01123
01 01125
01 01127
01 01129
01 01131
01 01133
02 02013
02 02016
02 02020
02 02050
02 02060
02 02063
02 02066
02 02068
02 02070
02 02090
02 02100
02 02105
02 02110
02 02122
02 02130
02 02150
02 02158
02 02164
02 02170
02 02180
02 02185
02 02188
02 02195
02 02198
02 02220
02 02230
02 02240
02 02275
02 02282
02 02290
04 04001
04 04003
04 04005
04 04007
04 04009
04 04011
04 04012
04 04013
04 04015
04 04017
04 04019
04 04021
04 04023
04 04025
0